In [1]:
import gc
import os
import random

import matplotlib.pyplot as plt

import lightgbm as lgb
from catboost import Pool, CatBoostRegressor
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, GroupKFold
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression, Ridge

# matplotlib and seaborn for plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import calendar
import datetime

path_data = "/kaggle/input/ashrae-energy-prediction/"
path_train = path_data + "train.csv"
path_test = path_data + "test.csv"
path_building = path_data + "building_metadata.csv"
path_weather_train = path_data + "weather_train.csv"
path_weather_test = path_data + "weather_test.csv"

myfavouritenumber = 666
seed = myfavouritenumber
random.seed(seed)

import xgboost as xgb
from xgboost import plot_importance, plot_tree

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
os.listdir('/kaggle/input/')

['submissions-v251',
 'ashrae-leak-data-station',
 'energy-demand-submission-mystratify',
 'ashrae-energy-prediction',
 'ashrae-simple-data-cleanup-lb-1-08-no-leaks',
 'ashrae-energy-prediction-using-stratified-kfold']

In [3]:
# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16 or not. feather format does not support float16.
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
df_test = pd.read_csv("/kaggle/input/ashrae-energy-prediction/" + "test.csv")
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00
...,...,...,...,...
41697595,41697595,1444,0,2018-05-09 07:00:00
41697596,41697596,1445,0,2018-05-09 07:00:00
41697597,41697597,1446,0,2018-05-09 07:00:00
41697598,41697598,1447,0,2018-05-09 07:00:00


In [5]:
%%time
submission_lgb_v251_1 = pd.read_csv("../input/submissions-v251/submission.csv")
submission_lgb_v251_2 = pd.read_csv("../input/submissions-v251/submission_lgb2_v251_2.csv")
submission_lgb_v251_3 = pd.read_csv("../input/submissions-v251/submission_lgb2_v251_3.csv")

CPU times: user 39.1 s, sys: 4.56 s, total: 43.7 s
Wall time: 51 s


In [6]:
%%time 
#DE OTROS
# entrena todo de una, sin folds
#submission_simple_cleanup = pd.read_csv("../input/ashrae-simple-data-cleanup-lb-1-08-no-leaks/submission.csv")
# stratify en tres grupos de meses
submission_stratify = pd.read_csv("../input/ashrae-energy-prediction-using-stratified-kfold/fe2_lgbm.csv")
# HLAF AND HALF
#submission_half = pd.read_csv("../input/ashrae-half-and-half/submission.csv")
#submission_divide_and_conquer = pd.read_csv('../input/ashrae-divide-and-conquer/submission_noleak.csv')

CPU times: user 13 s, sys: 1.76 s, total: 14.7 s
Wall time: 14.8 s


In [7]:
# mis stratifies
#submission_s3f = pd.read_csv('../input/energy-demand-submission-mystratify/submission_3folds.csv')
#submission_s5f = pd.read_csv('../input/energy-demand-submission-mystratify/submission_5folds.csv')

In [8]:
df_test['pred1'] = submission_lgb_v251_1.meter_reading
df_test['pred2'] = submission_lgb_v251_2.meter_reading
df_test['pred3'] = submission_lgb_v251_3.meter_reading
#df_test['pred4'] = submission_simple_cleanup.meter_reading #submission_s3f
df_test['pred5'] = submission_stratify.meter_reading
#df_test['pred6'] = submission_s5f.meter_reading
#df_test['pred4'] = submission_simple_cleanup.meter_reading
#df_test['pred6'] = submission_half.meter_reading

# limpiarmos
#del  submission_lgb_v251_1,  submission_lgb_v251_2,  submission_lgb_v251_3
gc.collect()

11

In [9]:
leak_df = pd.read_feather('../input/ashrae-leak-data-station/leak.feather')
leak_df.fillna(0, inplace=True)

leak_df = leak_df[(leak_df.timestamp.dt.year > 2016) & (leak_df.timestamp.dt.year < 2019)]
leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0 # remove large negative values
leak_df = leak_df[leak_df.building_id!=245]

df_test = reduce_mem_usage(df_test)
leak_df = reduce_mem_usage(leak_df)

Memory usage of dataframe is 2545.02 MB
Memory usage after optimization is: 1232.74 MB
Decreased by 51.6%
Memory usage of dataframe is 460.05 MB
Memory usage after optimization is: 299.03 MB
Decreased by 35.0%


In [10]:
print(leak_df.shape)

(12059979, 4)


In [11]:
leak_df = leak_df.merge(df_test[['building_id', 'meter', 'timestamp', 'pred1', 'pred2', 'pred3', 'pred5', 'row_id']],
                        left_on = ['building_id', 'meter', 'timestamp'], right_on = ['building_id', 'meter', 'timestamp'], how = "left")
#leak_df = leak_df.merge(building_meta_df[['building_id', 'site_id']], on='building_id', how='left')
del df_test
gc.collect()

0

In [12]:
scores = []
leak_score1 = np.sqrt(mean_squared_error(np.log1p(leak_df.pred1), np.log1p(leak_df.meter_reading)))
print ('RMSE pred1 ', leak_score1)
leak_score2 = np.sqrt(mean_squared_error(np.log1p(leak_df.pred2), np.log1p(leak_df.meter_reading)))
print ('RMSE pred2 ', leak_score2)
leak_score3 = np.sqrt(mean_squared_error(np.log1p(leak_df.pred3), np.log1p(leak_df.meter_reading)))
print ('RMSE pred3 ', leak_score3)
'''
leak_score4 = np.sqrt(mean_squared_error(np.log1p(leak_df.pred4), np.log1p(leak_df.meter_reading)))
print ('RMSE pred4 ', leak_score4)
'''
leak_score5 = np.sqrt(mean_squared_error(np.log1p(leak_df.pred5), np.log1p(leak_df.meter_reading)))
print ('RMSE pred5 ', leak_score5)
'''
leak_score6 = np.sqrt(mean_squared_error(np.log1p(leak_df.pred6), np.log1p(leak_df.meter_reading)))
print ('RMSE pred6 ', leak_score6)
'''
scores.append(leak_score1)
scores.append(leak_score2)
scores.append(leak_score3)
#scores.append(leak_score4)
scores.append(leak_score5)
#scores.append(leak_score6)

RMSE pred1  0.9856537
RMSE pred2  0.9967132
RMSE pred3  1.0349756
RMSE pred5  0.97505987


In [13]:
# calculando pesos a partir del score
def calculate_weigths_inverse(lista_scores):
    
    lista_scores = np.array(lista_scores)

    weights = 100 - lista_scores
    total_score = np.sum(weights)

    weights = weights/total_score

    return weights

list_scores = calculate_weigths_inverse(scores)
list_scores
#[0.3334011054973521, 0.3333638658729094, 0.3332350286297385]

array([0.2500314 , 0.25000352, 0.24990688, 0.25005817], dtype=float32)

In [14]:
print(leak_df.shape)

(12059979, 9)


In [15]:
leak_df.head(15)

,building_id,meter,meter_reading,timestamp,pred1,pred2,pred3,pred5,row_id
0,0,0.0,173.370300,2017-01-01,187.446701,181.708313,223.225952,166.884735,0
1,1,0.0,53.512718,2017-01-01,85.123627,74.787704,101.238258,77.057228,1
2,2,0.0,6.143042,2017-01-01,7.201028,7.030732,8.219783,7.092007,2
3,3,0.0,101.701469,2017-01-01,303.786530,301.138519,141.428040,296.035858,3
4,4,0.0,1141.240723,2017-01-01,1329.340210,1392.182739,1360.695557,1365.723999,4
5,5,0.0,13.651204,2017-01-01,11.113725,10.828140,56.259232,9.786079,5
6,6,0.0,260.874512,2017-01-01,118.102547,125.684387,78.346298,133.258820,6
7,7,0.0,502.705597,2017-01-01,410.038452,400.102173,458.879791,466.552856,7
8,7,1.0,717.574036,2017-01-01,52.620213,66.405609,928.632446,775.338623,8
9,8,0.0,513.285278,2017-01-01,418.487640,393.676300,275.443268,395.379364,9


In [16]:
%%time
from sklearn.ensemble import RandomForestRegressor

X = np.column_stack([leak_df.pred1, leak_df.pred2, leak_df.pred5]) #leak_df.pred3,leak_df.pred2,, leak_df.pred6
# y = 1 * x_0 + 2 * x_1 + 3
y = leak_df['meter_reading']
#reg = RandomForestRegressor(verbose=2).fit(X, y)
reg = LinearRegression().fit(X, y)
reg.score(X, y)

CPU times: user 1.4 s, sys: 860 ms, total: 2.26 s
Wall time: 1.42 s


0.16158910968852624

In [17]:
c = calculate_weigths_inverse(reg.coef_)
#c = calculate_weigths_inverse(reg.feature_importances_)
#c = list_scores
c

array([0.3349533 , 0.3332237 , 0.33182296], dtype=float32)

In [18]:
resultado_ponderado = c[0] * submission_lgb_v251_1.meter_reading +\
c[1] * submission_lgb_v251_2.meter_reading + c[2] * submission_stratify.meter_reading #+\ submission_lgb_v251_3
#c[3] * submission_simple_cleanup.meter_reading + c[4] * submission_stratify.meter_reading
#c[2] * submission_lgb_v251_3.meter_reading +\
#c[3] * submission_s3f.meter_reading +\
#c[4] * submission_stratify.meter_reading #+\
#c[5] * submission_s5f.meter_reading 
#c[4] * submission_stratify.meter_reading #+\
#c[5] * submission_simple_cleanup.meter_reading 

resultado_ponderado_leak = c[0] * leak_df.pred1 +\
c[1] * leak_df.pred2 + c[2] * leak_df.pred5 #+ c[3] * leak_df.pred4+ c[4] * leak_df.pred5
#c[2] * leak_df.pred3 +\
#c[3] * leak_df.pred4 +\
#c[4] * leak_df.pred5 #+\
#c[5] * leak_df.pred6

'''
media_modelos = (submission_lgb_v251_1.meter_reading + submission_lgb_v251_2.meter_reading +\
                 submission_lgb_v251_3.meter_reading  + submission_s3f.meter_reading +\
                submission_stratify.meter_reading  + submission_s5f.meter_reading) / 6 
'''
media_modelos = (submission_lgb_v251_1.meter_reading + submission_lgb_v251_2.meter_reading + submission_stratify.meter_reading
                ) / 3 
#submission_simple_cleanup.meter_reading + submission_stratify.meter_reading
#submission_simple_cleanup.meter_reading#+  + submission_stratify.meter_reading , submission_lgb_v251_2
#media_modelos_leak = (leak_df.pred1 + leak_df.pred2 + leak_df.pred3 + leak_df.pred4 + leak_df.pred5 + leak_df.pred6 ) / 6 
media_modelos_leak = (leak_df.pred1 + leak_df.pred2 +  + leak_df.pred5) / 3
#leak_df.pred4 ++ leak_df.pred6

In [19]:
leak_score = np.sqrt(mean_squared_error(np.log1p(resultado_ponderado_leak), np.log1p(leak_df.meter_reading)))
print ('RMSE resultado ponderado ', leak_score)

RMSE resultado ponderado  0.9519578


In [20]:
leak_score = np.sqrt(mean_squared_error(np.log1p(media_modelos_leak), np.log1p(leak_df.meter_reading)))
print ('RMSE media modelos ', leak_score)

RMSE media modelos  0.95198154


In [21]:
#leak_score = np.sqrt(mean_squared_error(np.log1p(mediana_modelos_leak), np.log1p(leak_df.meter_reading)))
#print ('RMSE mediana modelos ', leak_score)

In [22]:
submission = pd.DataFrame({"row_id": submission_lgb_v251_1.row_id, "meter_reading": resultado_ponderado})
submission.head(15)

,row_id,meter_reading
0,0,178.711595
1,1,79.002835
2,2,7.108105
3,3,300.332299
4,4,1362.353793
5,5,10.578017
6,6,125.658192
7,7,425.480216
8,8,297.028392
9,9,402.552039


In [23]:
submission.to_csv("submission.csv", index=False)

In [24]:
leak_df = leak_df[['meter_reading', 'row_id']].set_index('row_id').dropna()
submission.loc[leak_df.index, 'meter_reading'] = leak_df['meter_reading']

In [25]:
submission.to_csv("submission_leaked.csv", index=False)